In [37]:

include("../src/plasm.jl")
include("../src/read.jl")
include("../src/surface.jl")
include("../src/representation.jl")
include("../src/import3d.jl")

using PyCall

In [20]:

@pyimport io3d

In [21]:
data = io3d.read("~/projects/io3d/sample_data/jatra_5mm/", dataplus_format=true)
data3d  = data["data3d"]
voxelsize_mm = data["voxelsize_mm"]

3-element Array{Any,1}:
 5.0 
 0.75
 0.75

In [22]:
data

Dict{Any,Any} with 9 entries:
  "voxelsize_mm"      => Any[5.0,0.75,0.75]
  "data3d"            => Int16[-1024 -1024 … -1024 -1024; -1024 -1024 … -1024 -…
  "datadir"           => "/home/mjirik/projects/io3d/sample_data/jatra_5mm"
  "Modality"          => "CT"
  "ImageComments"     => ""
  "SeriesNumber"      => 7
  "series_number"     => 7
  "dcmfilelist"       => Any["/home/mjirik/projects/io3d/sample_data/jatra_5mm/…
  "SeriesDescription" => "DE_Abdom_1F  5.0  B30f M_0.3"

In [23]:
data3d = data3d[:, 100:150, 100:140]

93×51×41 Array{Int16,3}:
[:, :, 1] =
   -70    -18    -37    -71    -82  …    -65    -68    -60    -50    -37
   -21    -49    -77    -86    -94        -6    -17    -16    -29    -54
   -71    -72    -82    -90    -97        52     52     44     38     53
   -81    -77    -74    -78    -83       -38    -46    -43    -27      1
   -70    -76    -74    -83    -97        43     70     66     62     69
   -93    -87    -96    -88    -90  …     13     11      8     -1    -17
  -107   -111   -105    -94    -93        -8    -63   -131   -218   -320
   -81   -100   -132    -99    -89      -356   -509   -630   -752   -829
   -95    -99    -96   -100   -114      -468   -614   -726   -812   -873
   -96   -112   -105    -97    -91      -255   -394   -509   -573   -654
  -108   -119   -113   -108   -105  …   -312   -360   -308   -209   -187
   -94   -102   -114   -117   -113       -82    -94    -54      3     42
  -102   -114   -108    -84    -73        47     38     41     38     57
     ⋮        

In [39]:
import_data3d(data3d, voxelsize_mm, threshold)

step1
step2
step3
Removed face [9,28][51 50 46 47; 30 26 46 50; 31 30 50 51; 47 46 26 27; 51 47 27 31; 31 27 26 30; 71 70 66 67; 50 46 66 70; 51 50 70 71; 67 66 46 47; 71 67 47 51; 51 47 46 50; 91 90 86 87; 70 66 86 90; 71 70 90 91; 87 86 66 67; 91 87 67 71; 71 67 66 70; 55 54 50 51; 34 30 50 54; 35 34 54 55; 51 50 30 31; 55 51 31 35; 35 31 30 34; 75 74 70 71; 54 50 70 74; 55 54 74 75; 71 70 50 51; 75 71 51 55; 55 51 50 54; 95 94 90 91; 74 70 90 94; 75 74 94 95; 91 90 70 71; 95 91 71 75; 75 71 70 74]9
Removed face [1,12][51 50 46 47; 30 26 46 50; 31 30 50 51; 47 46 26 27; 51 47 27 31; 31 27 26 30; 71 70 66 67; 50 46 66 70; 51 50 70 71; 67 66 46 47; 71 67 47 51; 51 47 46 50; 91 90 86 87; 70 66 86 90; 71 70 90 91; 87 86 66 67; 91 87 67 71; 71 67 66 70; 55 54 50 51; 34 30 50 54; 35 34 54 55; 51 50 30 31; 55 51 31 35; 35 31 30 34; 75 74 70 71; 54 50 70 74; 55 54 74 75; 71 70 50 51; 75 71 51 55; 55 51 50 54; 95 94 90 91; 74 70 90 94; 75 74 94 95; 91 90 70 71; 95 91 71 75; 75 71 70 74]1
Remo

LoadError: UndefVarError: triface not defined

In [24]:
## Artifical data
data3d = zeros(Int16,5,4,3)
data3d[2:4,2:3,2] = 10
data3d

5×4×3 Array{Int16,3}:
[:, :, 1] =
 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  0  0  0

[:, :, 2] =
 0   0   0  0
 0  10  10  0
 0  10  10  0
 0  10  10  0
 0   0   0  0

[:, :, 3] =
 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  0  0  0
 0  0  0  0

In [25]:
# useless

function thresholding(data3d, threshold)
    sz = size(data3d)
    segmentation = Array(Int8, size(data3d, 1), size(data3d, 2), size(data3d, 3))
    # produce vertexes
    for k in 1:sz[3]
        for j in 1:sz[2]
            for i in 1:sz[1]
                if data3d[i, j, k] > 0

                    segmentation[i, j, k] = data3d[i, j, k] > threshold
    #             if i == 0
    #             index = 1 + i + sz[1] * j + sz[1] * sz[2] * k
                end

            end
        end
    end
    return segmentation
end

thresholding (generic function with 1 method)

In [26]:
threshold = 0

0

## Get vertices and count the voxels (* 4 = face number)

In [27]:
# Get all vertices and count the voxels
function get_index(sz, i, j, k) 
    index = 1 + i + (1 + sz[1]) * j + (sz[1] + 1) * (sz[2] + 1) * k 
    return index
end

function vertices_and_count_voxels_as_the_square_face_preprocessing(data3d, voxelsize_mm, threshold=0)
    """
    
    Fill the area with all possible vertices. Most of them will be removed in future.
    """
    threshold = 0
    ivertex = 0
    ifaces = 0
    sz = size(data3d)
    verts = Array(Float64, (sz[1]+1) * (sz[2]+1) * (sz[3]+1), 3)
    # produce vertexes
    nvoxels = 0
    for k in 0:sz[3]
        for j in 0:sz[2]
            for i in 0:sz[1]
                if (i > 0) & (j > 0) & (k > 0)
                    if data3d[i,j,k] > threshold
                        nvoxels += 1
                    end
                end
                pt = get_index(sz, i - 0, j -0, k -0)
    #             println(pt)
                verts[pt, 1] = i * voxelsize_mm[1]
                verts[pt, 2] = j * voxelsize_mm[2]
                verts[pt, 3] = k * voxelsize_mm[3]
            end
        end
    end
    return verts, nvoxels
end


#
#      
#       14 -------- 13
#     / |          / |
#   /   |        /   |          
# 11 --------- 12    |
# |     |       |    |        i 
# |     |       |    |               
# |     24 -----|-- 23        |     
# |   /         |   /         |   /
# | /           | /           | /
# 21 --------- 22             * ------ 
#
#
function create_square_faces(data3d, nvoxels, threshold=0)
    """
    create square faces based on vertices from vertices_and_count_voxels_as_...
    :nvoxels: number of voxels higher than the threslold.
    """
    nfaces = nvoxels * 6
    faces = Array(Int64, nfaces, 4)
    sz = size(data3d)
    ifaces = 0
#     println(sz)
    # produce faces
    for k in 1:sz[3]
        for j in 1:sz[2]
            for i in 1:sz[1]
    #             println(i,",",j,",",k)
                if data3d[i,j,k] > threshold
    #                 println("jsme uvnitr")
                    pt11 = get_index(sz, i -0, j -0, k -0)
    #                 println("jsme uvnitr", pt11)
                    pt12 = get_index(sz, i -0, j -0, k -1)
                    pt13 = get_index(sz, i -0, j -1, k -1)
                    pt14 = get_index(sz, i -0, j -1, k -0)
                    pt21 = get_index(sz, i -1, j -0, k -0)
                    pt22 = get_index(sz, i -1, j -0, k -1)
                    pt23 = get_index(sz, i -1, j -1, k -1)
                    pt24 = get_index(sz, i -1, j -1, k -0)

                    faces[ifaces + 1, :] = [pt11, pt12, pt13, pt14]'
                    faces[ifaces + 2, :] = [pt22, pt23, pt13, pt12]
                    faces[ifaces + 3, :] = [pt21, pt22, pt12, pt11]
                    faces[ifaces + 4, :] = [pt14, pt13, pt23, pt24]
                    faces[ifaces + 5, :] = [pt11, pt14, pt24, pt21]
                    faces[ifaces + 6, :] = [pt21, pt24, pt23, pt22]
                    ifaces += 6

                end
            end
        end
    end
    return faces
end

function arrayofarray2arrayd2d(d)
    data = Array(Float64,length(d),length(d[1]))
    for i in 1:length(d)
        for j in 1:length(d[1])
            data[i,j] = d[i][j]
        end
    end
    return data
end
function keep_surface_faces(vertexes, faces, index_base=0)
#     removeDoubleVertexesAndFaces
    println("step1")
    new_vertexes, inv_vertexes = remove_double_vertexes_alternative(vertexes)# , index_base=index_base)
    println("step2")
    new_faces = reindexVertexesInFaces(faces, inv_vertexes)
    println("step3")
    new_faces = removeDoubleFacesByAlberto(faces)
    println("step1")
    # todo remove unused vertexes
#     new_new_vertexes, new_new_inv_vertexes = remove_double_vertexes_alternative(new_vertexes)# , index_base=index_base)
    println("step2")
#     new_faces = reindexVertexesInFaces(faces, inv_vertexes)
    new_vertexes = arrayofarray2arrayd2d(new_vertexes)
    return new_vertexes, new_faces
end
#             if segmentation[i,j,k]
function triangulation(faces)
    trifaces = Array(Int64, size(faces, 1) * 2, 3)
    for i in 1:size(faces,1)
        face = faces[i,:]
        trifaces[(i * 2) - 0, :] = [face[1], face[2], face[3]]
        trifaces[(i * 2) - 1, :] = [face[1], face[3], face[4]]
    end
    return trifaces
end

triangulation (generic function with 1 method)

In [28]:
verts, nvoxels = vertices_and_count_voxels_as_the_square_face_preprocessing(data3d, voxelsize_mm, threshold)
# get_index(sz, 0, 0, 1)

(
[0.0 0.0 0.0; 5.0 0.0 0.0; … ; 20.0 3.0 2.25; 25.0 3.0 2.25],

6)

In [29]:
verts

120×3 Array{Float64,2}:
  0.0  0.0   0.0 
  5.0  0.0   0.0 
 10.0  0.0   0.0 
 15.0  0.0   0.0 
 20.0  0.0   0.0 
 25.0  0.0   0.0 
  0.0  0.75  0.0 
  5.0  0.75  0.0 
 10.0  0.75  0.0 
 15.0  0.75  0.0 
 20.0  0.75  0.0 
 25.0  0.75  0.0 
  0.0  1.5   0.0 
  ⋮              
  0.0  2.25  2.25
  5.0  2.25  2.25
 10.0  2.25  2.25
 15.0  2.25  2.25
 20.0  2.25  2.25
 25.0  2.25  2.25
  0.0  3.0   2.25
  5.0  3.0   2.25
 10.0  3.0   2.25
 15.0  3.0   2.25
 20.0  3.0   2.25
 25.0  3.0   2.25

## Create face

In [30]:
faces = create_square_faces(data3d, nvoxels, threshold)
# get_index(sz, 0,0,0)
faces

36×4 Array{Int64,2}:
 75  45  39  69
 44  38  39  45
 74  44  45  75
 69  39  38  68
 75  69  68  74
 74  68  38  44
 76  46  40  70
 45  39  40  46
 75  45  46  76
 70  40  39  69
 76  70  69  75
 75  69  39  45
 77  47  41  71
  ⋮            
 82  52  46  76
 51  45  46  52
 81  51  52  82
 76  46  45  75
 82  76  75  81
 81  75  45  51
 83  53  47  77
 52  46  47  53
 82  52  53  83
 77  47  46  76
 83  77  76  82
 82  76  46  52

## Double face removal

TODO

In [31]:
new_verts, new_faces = keep_surface_faces(verts, faces)

step1
step2
step3
Removed face [9,28][51 50 46 47; 30 26 46 50; 31 30 50 51; 47 46 26 27; 51 47 27 31; 31 27 26 30; 71 70 66 67; 50 46 66 70; 51 50 70 71; 67 66 46 47; 71 67 47 51; 51 47 46 50; 91 90 86 87; 70 66 86 90; 71 70 90 91; 87 86 66 67; 91 87 67 71; 71 67 66 70; 55 54 50 51; 34 30 50 54; 35 34 54 55; 51 50 30 31; 55 51 31 35; 35 31 30 34; 75 74 70 71; 54 50 70 74; 55 54 74 75; 71 70 50 51; 75 71 51 55; 55 51 50 54; 95 94 90 91; 74 70 90 94; 75 74 94 95; 91 90 70 71; 95 91 71 75; 75 71 70 74]9
Removed face [1,12][51 50 46 47; 30 26 46 50; 31 30 50 51; 47 46 26 27; 51 47 27 31; 31 27 26 30; 71 70 66 67; 50 46 66 70; 51 50 70 71; 67 66 46 47; 71 67 47 51; 51 47 46 50; 91 90 86 87; 70 66 86 90; 71 70 90 91; 87 86 66 67; 91 87 67 71; 71 67 66 70; 55 54 50 51; 34 30 50 54; 35 34 54 55; 51 50 30 31; 55 51 31 35; 35 31 30 34; 75 74 70 71; 54 50 70 74; 55 54 74 75; 71 70 50 51; 75 71 51 55; 55 51 50 54; 95 94 90 91; 74 70 90 94; 75 74 94 95; 91 90 70 71; 95 91 71 75; 75 71 70 74]1
Remo

(
[0.0 0.0 0.0; 0.0 0.0 0.75; … ; 25.0 3.0 1.5; 25.0 3.0 2.25],

[30 26 46 50; 50 46 66 70; … ; 31 27 26 30; 47 46 26 27])

In [32]:
new_verts

120×3 Array{Float64,2}:
  0.0  0.0   0.0 
  0.0  0.0   0.75
  0.0  0.0   1.5 
  0.0  0.0   2.25
  0.0  0.75  0.0 
  0.0  0.75  0.75
  0.0  0.75  1.5 
  0.0  0.75  2.25
  0.0  1.5   0.0 
  0.0  1.5   0.75
  0.0  1.5   1.5 
  0.0  1.5   2.25
  0.0  2.25  0.0 
  ⋮              
 25.0  1.5   0.0 
 25.0  1.5   0.75
 25.0  1.5   1.5 
 25.0  1.5   2.25
 25.0  2.25  0.0 
 25.0  2.25  0.75
 25.0  2.25  1.5 
 25.0  2.25  2.25
 25.0  3.0   0.0 
 25.0  3.0   0.75
 25.0  3.0   1.5 
 25.0  3.0   2.25

In [33]:
verts = new_verts
faces = new_faces

22×4 Array{Int64,2}:
 30  26  46  50
 50  46  66  70
 87  86  66  67
 70  66  86  90
 35  31  30  34
 75  74  94  95
 91  87  67  71
 71  67  47  51
 95  94  90  91
 54  50  70  74
 67  66  46  47
 51  47  27  31
 91  90  86  87
 55  54  74  75
 74  70  90  94
 95  91  71  75
 34  30  50  54
 55  51  31  35
 75  71  51  55
 35  34  54  55
 31  27  26  30
 47  46  26  27

## Triangulation

In [34]:
trifaces = triangulation(faces)

44×3 Array{Int64,2}:
 30  46  50
 30  26  46
 50  66  70
 50  46  66
 87  66  67
 87  86  66
 70  86  90
 70  66  86
 35  30  34
 35  31  30
 75  94  95
 75  74  94
 91  67  71
  ⋮        
 34  50  54
 34  30  50
 55  31  35
 55  51  31
 75  51  55
 75  71  51
 35  54  55
 35  34  54
 31  26  30
 31  27  26
 47  26  27
 47  46  26

## Visualization

In [35]:
check_vf(verts, trifaces)
visualize_numbers(to_lar(verts, trifaces), 2)

vertex index index higher than voxel number
faces min 26, faces max 95, vertex number 120
30
46
50
50
66
66
70
70
30
30
75
94
67
87
47
71
90
95
54
50
47
66
27
51
86
90
74
54
94
70
75
91
50
30
31
51
55
71
54
35
26
31
27
46
XgeModule::init_started
Creating shared GLCanvas...
shared GLCanvas created
XgeModule::init_finished
Evaluating fenvs.py..
('...fenvs.py imported in', 0.005560000000000009, 'seconds')
centre
centre
centre
Building batches from Hpc....
...done in 27 msec
Optimizing the octree....
   Number of input batches 3487
   total number vertices    232
   Number of output batches 116
   Batch vertex media       2
...done in 10 msec
Building octree from 116 batches....
Scene number of nodes of the octree 105
Scene max depth                     4
Scene number of batches             116
...done in 0 msec


In [36]:
uuu = [1,2]
size(uuu,1)

2